In [1]:
import voxel_slam

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
import numpy as np
import mrob 
import open3d as o3d 

import pathlib

### I/O 

In [3]:
def read_depth_to_rgb_map(association_path):
    depth_to_rgb_map = dict()
    with open(association_path) as data:
        for line in data:
            _, color_name, _, depth_name = line.split()
            color_name = color_name.strip('\n').lstrip("color/")
            depth_name = depth_name.strip('\n').lstrip("depth/")
            depth_to_rgb_map[depth_name] = color_name
    return depth_to_rgb_map

def parse_trajectories(input_path):
    ts = dict()
    with open(input_path) as data:
        for line in data:
            line_tokens = line.strip('\n').split()
            
            timestamp = line_tokens[0].replace('.', '')
            timestamp = "0" * (12 - len(timestamp)) + timestamp
            
            trajectory = np.asarray(list(map(float, line_tokens[1:])))
            ts.update({timestamp: trajectory})
    return ts


def filename_to_pose(depth_filename: str, depth_to_rgb_map: str, poses: dict):
    rgb_filename = depth_to_rgb_map[depth_filename].rstrip(".png")
    return poses["0" * (12 - len(rgb_filename)) + rgb_filename]


def trajectory_to_se3(trajectory):
    t, Q = trajectory[:3], trajectory[3:]
    R = mrob.geometry.SO3(mrob.geometry.quat_to_so3(Q))
    return mrob.geometry.SE3(R, t)

def parse_trajectories_float(input_path, ts_multiplier=1e9):
    ts = dict()
    with open(input_path) as data:
        for line in data:
            line_tokens = line.strip('\n').split()
            
            timestamp = float(line_tokens[0]) * ts_multiplier
            
            trajectory = np.asarray(list(map(float, line_tokens[1:])))
            ts.update({timestamp: trajectory})
    return ts

In [4]:
def read_hilti_sequence(ts_to_quat, ts_to_depth_path, start_of_sequence=0, number_of_clouds=-1, h=1.5):
    poses = []
    clouds = []

    lidar_so3 = mrob.geometry.SO3(mrob.geometry.quat_to_so3(np.asarray([ 0.7071068, -0.7071068, 0, 0 ])))
    lidat_t = np.asarray([ -0.001, -0.00855, 0.055 ])   
    imu_to_lidar_se3 = mrob.geometry.SE3(lidar_so3, lidat_t).T()

    for ts in sorted(ts_to_quat)[start_of_sequence : start_of_sequence + number_of_clouds]:
        imu_pose = trajectory_to_se3(ts_to_quat[ts]).T()

        pose = imu_pose @ imu_to_lidar_se3
        cloud = o3d.io.read_point_cloud(str(ts_to_depth_path[ts]))
        cloud_points = np.asarray(cloud.points)
        f = np.where(np.linalg.norm(cloud_points, axis=1) > h)[0] 

        cloud.points = o3d.utility.Vector3dVector(cloud_points[f])

        cloud.paint_uniform_color([0.0, 0.0, 0.0])
        
        poses.append(pose)
        clouds.append(cloud)

    return clouds, poses

## VoxelSLAM

In [5]:
def get_plane_mse(points, verbose=False):
    c = np.mean(points, axis=0)
    A = np.array(points) - c
    eigvals, _ = np.linalg.eig(A.T @ A)
    if verbose:
        print(min(eigvals) / points.shape[0])
    return min(eigvals) / points.shape[0]

In [6]:
import copy

In [7]:
def get_optimized_poses(clouds_cp, poses_cp):
    clouds = copy.deepcopy(clouds_cp)
    poses = copy.deepcopy(poses_cp)
    voxel_map = voxel_slam.frontend.VoxelFeatureMap(clouds, poses, voxel_size=2)
    f_function = lambda points: np.asarray(points).shape[0] > 0
    
    voxel_feature_map = voxel_map.extract_voxel_features(
        ransac_distance_threshold=0.005, 
        points_filter_function=f_function
    )
    voxel_map.filter_voxel_features(voxel_feature_map, preserve_non_informative_voxels=True)
    graph = voxel_slam.backend.FGraph(voxel_feature_map, number_of_poses=len(poses))
    optimized_poses = graph.get_optimized_poses()
    colored_pcd, color_to_voxel_id = voxel_map.get_colored_feature_clouds(voxel_feature_map, color_method="voxel")
    # return color_to_voxel_id
    return colored_pcd, optimized_poses

In [8]:
clouds_path = "/home/ach/Desktop/datasets/hilti/out2"
poses_path = "/home/ach/Desktop/datasets/hilti/exp14_basement_2_imu.txt"

In [9]:
ts_multiplier = 1 / 1e9
ts_to_depth_path = {float(x.stem) * ts_multiplier : x for x in pathlib.Path(clouds_path).iterdir()}
ts_to_quat = parse_trajectories_float(poses_path, ts_multiplier)

In [10]:
clouds, poses = read_hilti_sequence(ts_to_quat, ts_to_depth_path, start_of_sequence=300, number_of_clouds=30, h=3)
# clouds, poses = read_hilti_sequence(ts_to_quat, ts_to_depth_path, start_of_sequence=0, number_of_clouds=400, h=3)

In [11]:
o3d.visualization.draw_geometries([voxel_slam.aggregate_map(clouds, poses, enable_color=True)])

### AHC Optimizer

In [12]:
window_size = 5
optimized_submaps = {}

transformed_clouds = voxel_slam.VoxelFeatureMap(clouds, poses, voxel_size=2).transformed_clouds

for i in range(0, len(clouds), window_size):
    optimized_submaps[i] = get_optimized_poses(transformed_clouds[i:i+window_size], [np.eye(4) for _ in range(window_size)])

# for i in range(0, len(clouds), window_size):
#     optimized_submaps[i] = get_optimized_poses(clouds[i:i+window_size], poses[i:i+window_size])

FGraph initial error: 0.38892875661661935
Chi2: 0.5685292757284053
Time (init): 0.000804095
FGraph initial error: 22.311887878434383
Chi2: 15.122397632380487
Time (init): 0.00070744
FGraph initial error: 200.3095820687082
Chi2: 25.126707025089104
Time (init): 0.000824724
FGraph initial error: 210.72679229807403
Chi2: 9.803795503478298
Time (init): 0.000938893
FGraph initial error: 143.13762547866756
Chi2: 8.89666277206752
Time (init): 0.00146686
FGraph initial error: 91.15382819232799
Chi2: 50.91058038299922
Time (init): 0.004372985


In [13]:
aggregated_submaps = {k: voxel_slam.aggregate_map(pcd, pose) for k, (pcd, pose) in optimized_submaps.items()}

In [14]:
for k, pcd in aggregated_submaps.items():
    pcd.paint_uniform_color([0.0, 0.0, 0.0])

In [15]:
o3d.visualization.draw_geometries(list(aggregated_submaps.values()))

In [18]:
ahc_clouds, ahc_poses = get_optimized_poses(list(aggregated_submaps.values()), [np.eye(4) for _ in range(len(aggregated_submaps))])

FGraph initial error: 3814.903836350763
Chi2: 367.5497820987035
Time (init): 0.008787516


In [17]:
o3d.visualization.draw_geometries([voxel_slam.aggregate_map(ahc_clouds, ahc_poses)])

### Default pipeline

In [11]:
voxel_map = voxel_slam.frontend.VoxelFeatureMap(clouds, poses, voxel_size=2)

In [12]:
voxel_feature_map = voxel_map.extract_voxel_features(ransac_distance_threshold=0.003, points_filter_function=lambda points: np.asarray(points).shape[0] > 0)

In [13]:
voxel_map.filter_voxel_features(voxel_feature_map)

In [14]:
graph = voxel_slam.backend.FGraph(voxel_feature_map, number_of_poses=len(poses))
optimized_poses = graph.get_optimized_poses()

FGraph initial error: 13304.708160288104
Chi2: 1760.4497651593927


In [20]:
colored_clouds_voxel, color_to_voxel_id = voxel_map.get_colored_feature_clouds(voxel_feature_map, color_method="voxel")

Time (init): 0.007771299


In [21]:
optimized_map = voxel_slam.utility.aggregate_map(colored_clouds_voxel, optimized_poses)

In [22]:
o3d.visualization.draw_geometries([optimized_map])

In [18]:
def pick_points(pcd):
    vis = o3d.visualization.VisualizerWithEditing()
    vis.create_window()
    vis.add_geometry(pcd)
    vis.run()  # user picks points
    vis.destroy_window()
    print("")
    return vis.get_picked_points()

In [19]:
def get_points_stat():
    picked_points_idx = pick_points(optimized_map)
    pcd_fragment = optimized_map.select_by_index(picked_points_idx) 
    fragment_color = np.asarray(pcd_fragment.colors)[0]
    voxel_id = color_to_voxel_id[tuple(fragment_color)]

    keys = list(voxel_feature_map[voxel_id].keys())
    plane_points = voxel_feature_map[voxel_id][keys[0]].points
    o3d.visualization.draw_geometries([o3d.geometry.PointCloud(o3d.utility.Vector3dVector(plane_points))])

    c = np.mean(plane_points, axis=0)
    A = np.array(plane_points) - c
    eigvals, _ = np.linalg.eig(A.T @ A)
    eigvals.sort()

    print(eigvals)
    print("ev1 / ev2:", eigvals[1] / eigvals[2])
    print("min(ev) / sum(ev)", min(eigvals) / sum(eigvals))
    print("min(eigvals) / points.shape[0]", min(eigvals) / np.asarray(plane_points).shape[0])